In [1]:
import xgboost as xgb
import pandas as pd
import databento_sqlmod1 as dbs
#import databento_sql as dbs
import fmp_keymetrics
import combiner
import numpy as np
#import talib as ta  # ta-lib or ta for calculating technical indicators
import ta
from datetime import datetime
from tqdm import tqdm

In [2]:
ticker_list = ['AAPL', 'MSFT', 'NVDA',  # Technology
    'AMZN', 'TSLA', 'HD',  # Consumer Discretionary
    'UNH', 'JNJ', 'LLY',  # Healthcare
    'JPM', 'BAC', 'WFC',  # Financials
    'XOM', 'CVX', 'COP',  # Energy
    'PG', 'KO', 'PEP',  # Consumer Staples
    'BA', 'CAT', 'UPS',  # Industrials
    'LIN', 'APD', 'SHW',  # Materials
    'PLD', 'AMT', 'CCI',  # Real Estate
    'NEE', 'DUK', 'SO',  # Utilities
]

In [ ]:
# Old prepdata function
'''
def prepdata(ticker, start_date, end_date):
    # Retrieve data for the ticker and date range
    data = comb.get_comb_data_from_postgresql(ticker, start_date, end_date)

    # Create a DataFrame
    df = pd.DataFrame(data)

    # Sort the data by date
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(by='date')

    # Add technical indicators
    df = add_technical_indicators(df)

    # Calculate log returns as the target variable
    df['log_return'] = np.log(df['close'] / df['close'].shift(1))

    # Drop rows with NaN values in log returns and technical indicators
    df = df.dropna()

    # Define the fundamental columns and technical indicators to use as features
    fundamental_columns = [
        'revenuePerShare', 'netIncomePerShare', 'operatingCashFlowPerShare', 'freeCashFlowPerShare', 
        'cashPerShare', 'bookValuePerShare', 'tangibleBookValuePerShare', 'shareholdersEquityPerShare', 
        'interestDebtPerShare', 'marketCap', 'enterpriseValue', 'peRatio', 'priceToSalesRatio', 
        'pocfratio', 'pfcfRatio', 'pbRatio', 'ptbRatio', 'evToSales', 'enterpriseValueOverEBITDA', 
        'evToOperatingCashFlow', 'evToFreeCashFlow', 'earningsYield', 'freeCashFlowYield', 
        'debtToEquity', 'debtToAssets', 'netDebtToEBITDA', 'currentRatio', 'interestCoverage', 
        'incomeQuality', 'dividendYield', 'payoutRatio', 'salesGeneralAndAdministrativeToRevenue', 
        'researchAndDdevelopementToRevenue', 'intangiblesToTotalAssets', 'capexToOperatingCashFlow', 
        'capexToRevenue', 'capexToDepreciation', 'stockBasedCompensationToRevenue', 'grahamNumber', 
        'roic', 'returnOnTangibleAssets', 'grahamNetNet', 'workingCapital', 'tangibleAssetValue', 
        'netCurrentAssetValue', 'investedCapital', 'averageReceivables', 'averagePayables', 
        'averageInventory', 'daysSalesOutstanding', 'daysPayablesOutstanding', 
        'daysOfInventoryOnHand', 'receivablesTurnover', 'payablesTurnover', 'inventoryTurnover', 
        'roe', 'capexPerShare'
    ]
    
    # Technical columns added
    technical_columns = [
        'sma_20', 'sma_50', 'ema_20', 'rsi_14', 'macd', 'macd_signal', 'bb_upper', 'bb_lower', 
        'atr_14', 'obv', 'stoch_k', 'stoch_d', 'williams_r', 'cmf', 'ema_50', 'ema_100', 'ema_200'
    ]

    # Fill missing values in the fundamental columns using forward fill, followed by zeros if necessary
    df[fundamental_columns + technical_columns] = df[fundamental_columns + technical_columns].ffill().fillna(0)

    # Define features (X) and target (y)
    X = df[fundamental_columns + technical_columns]
    y = df['log_return']

    # Convert features into an XGBoost DMatrix
    dmat = xgb.DMatrix(X, label=y, missing=np.nan)

    return dmat, y, X'''

In [3]:
# Databento download
for ticker in ticker_list:
    dbs.download_and_append_data(ticker, '2019-01-01', '2023-12-31', frequency='daily')

#FMP Download
start_date = '2019-01-01'
end_date = '2024-09-31'

fmp = fmp_keymetrics.funda_ETL()
keymetrics = fmp.download_funda_data(ticker_list, start=start_date, end=end_date)
keymetrics

# Merge Databento and FMP data
comb = combiner.combine_pvol_funda()
#tickers = ['AAPL','AMZN','GOOGL','MSFT']
merged = comb.merge_pvol_funda(ticker_list)

# Initialize combiner instance
comb = combiner.combine_pvol_funda()

def add_technical_indicators(df):
    # Add SMA, EMA
    df['sma_20'] = ta.trend.sma_indicator(df['close'], window=20)
    df['sma_50'] = ta.trend.sma_indicator(df['close'], window=50)
    df['ema_20'] = ta.trend.ema_indicator(df['close'], window=20)
    df['ema_50'] = ta.trend.ema_indicator(df['close'], window=50)
    df['ema_100'] = ta.trend.ema_indicator(df['close'], window=100)
    df['ema_200'] = ta.trend.ema_indicator(df['close'], window=200)
    
    # Add RSI, MACD
    df['rsi_14'] = ta.momentum.rsi(df['close'], window=14)
    df['macd'] = ta.trend.macd(df['close'])
    df['macd_signal'] = ta.trend.macd_signal(df['close'])

    # Add Bollinger Bands
    bollinger = ta.volatility.BollingerBands(df['close'], window=20)
    df['bb_upper'] = bollinger.bollinger_hband()
    df['bb_lower'] = bollinger.bollinger_lband()
    
    # Add Average True Range (ATR)
    df['atr_14'] = ta.volatility.average_true_range(df['high'], df['low'], df['close'], window=14)
    
    # Add On-Balance Volume (OBV)
    df['obv'] = ta.volume.on_balance_volume(df['close'], df['volume'])
    
    # Add Stochastic Oscillator (K, D)
    stoch = ta.momentum.stoch(df['high'], df['low'], df['close'], window=14, smooth_window=3)
    df['stoch_k'] = stoch
    df['stoch_d'] = ta.momentum.stoch_signal(df['high'], df['low'], df['close'], window=14, smooth_window=3)
    
    # Remove momentum since it was causing an error
    # Add Williams %R
    df['williams_r'] = ta.momentum.williams_r(df['high'], df['low'], df['close'], lbp=14)
    
    # Add Chaikin Money Flow (CMF)
    df['cmf'] = ta.volume.chaikin_money_flow(df['high'], df['low'], df['close'], df['volume'], window=20)
    
    return df

def prepdata(ticker, start_date, end_date):
    # Retrieve data for the ticker and date range
    data = comb.get_comb_data_from_postgresql(ticker, start_date, end_date)

    # Create a DataFrame
    df = pd.DataFrame(data)

    # Sort the data by date
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(by='date')

    # Define the fundamental columns and technical indicators to use as features
    fundamental_columns = [
        'revenuePerShare', 'netIncomePerShare', 'operatingCashFlowPerShare', 'freeCashFlowPerShare', 
        'cashPerShare', 'bookValuePerShare', 'tangibleBookValuePerShare', 'shareholdersEquityPerShare', 
        'interestDebtPerShare', 'marketCap', 'enterpriseValue', 'peRatio', 'priceToSalesRatio', 
        'pocfratio', 'pfcfRatio', 'pbRatio', 'ptbRatio', 'evToSales', 'enterpriseValueOverEBITDA', 
        'evToOperatingCashFlow', 'evToFreeCashFlow', 'earningsYield', 'freeCashFlowYield', 
        'debtToEquity', 'debtToAssets', 'netDebtToEBITDA', 'currentRatio', 'interestCoverage', 
        'incomeQuality', 'dividendYield', 'payoutRatio', 'salesGeneralAndAdministrativeToRevenue', 
        'researchAndDdevelopementToRevenue', 'intangiblesToTotalAssets', 'capexToOperatingCashFlow', 
        'capexToRevenue', 'capexToDepreciation', 'stockBasedCompensationToRevenue', 'grahamNumber', 
        'roic', 'returnOnTangibleAssets', 'grahamNetNet', 'workingCapital', 'tangibleAssetValue', 
        'netCurrentAssetValue', 'investedCapital', 'averageReceivables', 'averagePayables', 
        'averageInventory', 'daysSalesOutstanding', 'daysPayablesOutstanding', 
        'daysOfInventoryOnHand', 'receivablesTurnover', 'payablesTurnover', 'inventoryTurnover', 
        'roe', 'capexPerShare'
    ]

    # Technical columns added
    technical_columns = [
        'sma_20', 'sma_50', 'ema_20', 'rsi_14', 'macd', 'macd_signal', 'bb_upper', 'bb_lower', 
        'atr_14', 'obv', 'stoch_k', 'stoch_d', 'williams_r', 'cmf', 'ema_50', 'ema_100', 'ema_200'
    ]

    # Add technical indicators
    df = add_technical_indicators(df)

    # Calculate log returns as the target variable
    df['log_return'] = np.log(df['close'] / df['close'].shift(1))

    # Fill missing values in the fundamental columns using forward fill, followed by backward fill, then zeros if necessary
    df[fundamental_columns + technical_columns] = df[fundamental_columns + technical_columns].ffill().bfill().fillna(0)

    # Drop rows with NaN values in log returns and technical indicators
    df = df.dropna()

    # Define features (X) and target (y)
    X = df[fundamental_columns + technical_columns]
    y = df['log_return']

    # Convert features into an XGBoost DMatrix
    dmat = xgb.DMatrix(X, label=y, missing=np.nan)

    return dmat, y, X


Ticker AAPL already up-to-date
data/equity/usa/daily/aapl.csv has been successfully zipped into data/equity/usa/daily/aapl.zip.
Data for AAPL fetched from postgres, converted to LEAN format.
Ticker MSFT already up-to-date
data/equity/usa/daily/msft.csv has been successfully zipped into data/equity/usa/daily/msft.zip.
Data for MSFT fetched from postgres, converted to LEAN format.
Ticker NVDA already up-to-date
data/equity/usa/daily/nvda.csv has been successfully zipped into data/equity/usa/daily/nvda.zip.
Data for NVDA fetched from postgres, converted to LEAN format.
Ticker AMZN already up-to-date
data/equity/usa/daily/amzn.csv has been successfully zipped into data/equity/usa/daily/amzn.zip.
Data for AMZN fetched from postgres, converted to LEAN format.
Ticker TSLA already up-to-date
data/equity/usa/daily/tsla.csv has been successfully zipped into data/equity/usa/daily/tsla.zip.
Data for TSLA fetched from postgres, converted to LEAN format.
Ticker HD already up-to-date
data/equity/usa/

In [4]:
# Example of how to use the updated function
#ticker_list = ['AAPL']
for ticker in tqdm(ticker_list, desc="Processing tickers", unit="ticker"):
    start_date = '2019-01-01'
    end_date = '2022-12-31'

    # Prepare the training data using log returns
    dtrain, y_train, X_train = prepdata(ticker, start_date, end_date)

    ''' Setting Test Data '''
    start_date = '2023-01-01'
    end_date = '2023-12-31'

    # Prepare the test data
    dtest, y_test, X_test = prepdata(ticker, start_date, end_date)

Processing tickers:   0%|          | 0/30 [00:00<?, ?ticker/s]

Processing tickers: 100%|██████████| 30/30 [00:08<00:00,  3.74ticker/s]


# Backtesting Framework

In [5]:
def log_trade(trade_log, ticker, date, position_type, trade_type, entry_price, exit_price=None, num_shares=None, signal=None, portfolio_value_start=None, portfolio_value_end=None):
    """
    Log trade details into the trade_log DataFrame.
    """
    return_on_trade = None
    if exit_price is not None and entry_price is not None:
        return_on_trade = (exit_price - entry_price) / entry_price

    trade_entry = pd.DataFrame({
        'ticker': [ticker],
        'date': [date],
        'position_type': [position_type],  # 'long' or 'short'
        'trade_type': [trade_type],  # 'buy' or 'sell'
        'entry_price': [entry_price],
        'exit_price': [exit_price],
        'return': [return_on_trade],
        'num_shares': [num_shares],  # Track the number of shares bought/sold
        'signal': [signal],
        'portfolio_value_start': [portfolio_value_start],  # Log portfolio value at start of trade
        'portfolio_value_end': [portfolio_value_end]  # Log portfolio value at end of trade
    })

    trade_log = pd.concat([trade_log, trade_entry], ignore_index=True)
    
    return trade_log

def calculate_portfolio_metrics(trade_log_df):
    """
    Calculate portfolio metrics based on the trade log.
    """
    if trade_log_df['return'].empty or trade_log_df['return'].isna().all():
        return {'Sharpe Ratio': float('nan'), 'Sortino Ratio': float('nan'), 'Max Drawdown': float('nan')}
    
    trade_returns = trade_log_df['return'].dropna()
    sharpe_ratio = (trade_returns.mean() / trade_returns.std()) * np.sqrt(252) if trade_returns.std() > 0 else float('nan')
    downside_risk = trade_returns[trade_returns < 0].std()
    sortino_ratio = (trade_returns.mean() / downside_risk) * np.sqrt(252) if downside_risk > 0 else float('nan')

    cumulative_return = (1 + trade_returns).cumprod()
    drawdown = cumulative_return / cumulative_return.cummax() - 1
    max_drawdown = drawdown.min()

    return {
        'Sharpe Ratio': sharpe_ratio,
        'Sortino Ratio': sortino_ratio,
        'Max Drawdown': max_drawdown
    }


In [28]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import joblib
from datetime import datetime

# Placeholder for ticker data (train and test)
ticker_list = ['AAPL', 'MSFT', 'NVDA', 'AMZN', 'TSLA', 'HD', 
               'UNH', 'JNJ', 'LLY', 'JPM', 'BAC', 'WFC', 
               'XOM', 'CVX', 'COP', 'PG', 'KO', 'PEP',
               'BA', 'CAT', 'UPS', 'LIN', 'APD', 'SHW', 
               'PLD', 'AMT', 'CCI', 'NEE', 'DUK', 'SO']

# Load the trained model
best_model = joblib.load('best_xgb_model.pkl')

# Initialize variables for backtesting
initial_cash = 100000  # Initial capital
cash = initial_cash
portfolio = {ticker: 0 for ticker in ticker_list}  # Track positions in each stock

# Initialize DataFrames
trade_log = pd.DataFrame(columns=['ticker', 'date', 'position_type', 'trade_type', 'entry_price', 'exit_price', 'return', 'signal', 'pnl'])
portfolio_value_df = pd.DataFrame(columns=['date', 'portfolio_value', 'cash_value', 'position_value'])

# Load your price data for 2022 and 2023
def get_data(ticker, start_date, end_date):
    """
    Fetch data from PostgreSQL; if missing, download from Databento.
    """
    df = dbs.get_data_from_postgresql(ticker, start_date, end_date)

    if df is None or df.empty:
        print(f"Data for {ticker} not found in PostgreSQL. Fetching from Databento.")
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
        df = dbs.get_data_from_databento(ticker, start_date, end_date)

        dbs.upload_to_postgresql(df, ticker)

    df['date'] = pd.to_datetime(df['ts_event'])
    df = df.sort_values(by='date')

    return df

# Cache price data for all tickers
data_cache = {ticker: get_data(ticker, '2023-01-01', '2023-12-31') for ticker in ticker_list}

# Generate signals for each ticker
signals = {}
for ticker in ticker_list:
    train_data = get_data(ticker, '2019-01-01', '2022-12-31')
    test_data = get_data(ticker, '2023-01-01', '2023-12-31')

    dtrain, y_train, X_train = prepdata(ticker, '2019-01-01', '2022-12-31')
    dtest, y_test, X_test = prepdata(ticker, '2023-01-01', '2023-12-31')

    # Ensure equal length for test data, X_test, and y_test
    min_len = min(len(test_data), len(X_test), len(y_test))
    test_data = test_data.iloc[:min_len]
    X_test = X_test[:min_len]
    y_test = y_test[:min_len]

    print(f"Test data shape: {test_data.shape}, X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

    y_pred = best_model.predict(X_test)
    # Ensure equal length for test data and predictions
    signals[ticker] = pd.DataFrame({'date': test_data['date'].values, 'predicted_change': y_pred})

# Backtesting loop day by day
unique_dates = sorted(set(signals[ticker_list[0]]['date']))
for current_date in unique_dates:
    position_value = 0
    for ticker in ticker_list:
        ticker_signal = signals[ticker]
        signal_data = ticker_signal[ticker_signal['date'] == current_date]
        if signal_data.empty:
            continue

        predicted_change = signal_data['predicted_change'].iloc[0]
        current_day_data = data_cache[ticker][data_cache[ticker]['date'] == current_date]
        if current_day_data.empty:
            continue

        next_open = current_day_data['open'].iloc[0]
        next_close = current_day_data['close'].iloc[0]
        signal = 'long' if predicted_change > 0 else 'short'

        # Update position values if no sell signal
        if portfolio[ticker] > 0 and predicted_change <= -0.01:
            exit_price = next_close
            entry_price = trade_log[trade_log['ticker'] == ticker]['entry_price'].iloc[-1]  # Get the last buy price
            trade_return = (exit_price - entry_price) / entry_price
            pnl = (exit_price - entry_price) * portfolio[ticker]

            # Log sell trade
            trade_log = pd.concat([trade_log, pd.DataFrame({
                'ticker': [ticker],
                'date': [current_date],
                'position_type': ['long'],
                'trade_type': ['sell'],
                'entry_price': [entry_price],
                'exit_price': [exit_price],
                'return': [trade_return],
                'signal': [signal],
                'portfolio_value_start': [entry_price * portfolio[ticker]],
                'portfolio_value_end': [exit_price * portfolio[ticker]],
                'pnl': [pnl]
            })], ignore_index=True)
            cash += portfolio[ticker] * exit_price
            portfolio[ticker] = 0

        # Buy signal
        if predicted_change > 0.01 and portfolio[ticker] == 0:
            num_shares = cash // next_open  # Integer division for full shares
            if num_shares > 0:
                portfolio[ticker] = num_shares
                cash -= num_shares * next_open

                # Log buy trade
                trade_log = pd.concat([trade_log, pd.DataFrame({
                    'ticker': [ticker],
                    'date': [current_date],
                    'position_type': ['long'],
                    'trade_type': ['buy'],
                    'entry_price': [next_open],
                    'exit_price': [np.nan],
                    'return': [np.nan],
                    'signal': [signal],
                    'portfolio_value_start': [next_open * num_shares],
                    'portfolio_value_end': [np.nan],
                    'pnl': [np.nan]
                })], ignore_index=True)

        # Update position value
        position_value += portfolio[ticker] * next_close

    # Calculate portfolio value daily and append to rolling DataFrame
    portfolio_value = position_value + cash
    portfolio_value_df = pd.concat([portfolio_value_df, pd.DataFrame({'date': [current_date], 'portfolio_value': [portfolio_value], 'cash_value': [cash], 'position_value': [position_value]})], ignore_index=True)

# Calculate final portfolio value
final_portfolio_value = cash + sum([portfolio[ticker] * data_cache[ticker][data_cache[ticker]['date'] == unique_dates[-1]]['close'].iloc[0] for ticker in ticker_list if not data_cache[ticker][data_cache[ticker]['date'] == unique_dates[-1]].empty])

print(f"Final Portfolio Value: {final_portfolio_value}")

# Save portfolio value log to CSV
portfolio_value_df.to_csv('portfolio_value_log.csv', index=False)

# Calculate portfolio performance metrics
def calculate_portfolio_metrics(portfolio_value_df):
    # Calculate daily returns from portfolio value changes
    portfolio_value_df['daily_return'] = portfolio_value_df['portfolio_value'].pct_change().fillna(0)

    sharpe_ratio = (portfolio_value_df['daily_return'].mean() / portfolio_value_df['daily_return'].std()) * np.sqrt(252)
    
    downside_risk = portfolio_value_df[portfolio_value_df['daily_return'] < 0]['daily_return'].std()
    sortino_ratio = (portfolio_value_df['daily_return'].mean() / downside_risk) * np.sqrt(252) if downside_risk > 0 else float('nan')

    # Max drawdown calculation
    cumulative_return = (1 + portfolio_value_df['daily_return']).cumprod()
    drawdown = cumulative_return / cumulative_return.cummax() - 1
    max_drawdown = drawdown.min()

    return {
        'Sharpe Ratio': sharpe_ratio if portfolio_value_df['daily_return'].std() > 0 else float('nan'),
        'Sortino Ratio': sortino_ratio,
        'Max Drawdown': max_drawdown
    }

# Recalculate portfolio metrics using the rolling portfolio value
portfolio_metrics = calculate_portfolio_metrics(portfolio_value_df)
print(portfolio_metrics)


Test data shape: (250, 11), X_test shape: (250, 74), y_test shape: (250,)
Test data shape: (249, 11), X_test shape: (249, 74), y_test shape: (249,)
Test data shape: (249, 11), X_test shape: (249, 74), y_test shape: (249,)
Test data shape: (249, 11), X_test shape: (249, 74), y_test shape: (249,)
Test data shape: (249, 11), X_test shape: (249, 74), y_test shape: (249,)
Test data shape: (249, 11), X_test shape: (249, 74), y_test shape: (249,)
Test data shape: (249, 11), X_test shape: (249, 74), y_test shape: (249,)
Test data shape: (250, 11), X_test shape: (250, 74), y_test shape: (250,)
Test data shape: (249, 11), X_test shape: (249, 74), y_test shape: (249,)
Test data shape: (249, 11), X_test shape: (249, 74), y_test shape: (249,)
Test data shape: (249, 11), X_test shape: (249, 74), y_test shape: (249,)
Test data shape: (249, 11), X_test shape: (249, 74), y_test shape: (249,)
Test data shape: (249, 11), X_test shape: (249, 74), y_test shape: (249,)
Test data shape: (249, 11), X_test sha

/tmp/ipykernel_4975/230901958.py:140: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  portfolio_value_df = pd.concat([portfolio_value_df, pd.DataFrame({'date': [current_date], 'portfolio_value': [portfolio_value], 'cash_value': [cash], 'position_value': [position_value]})], ignore_index=True)


Final Portfolio Value: 100000
{'Sharpe Ratio': nan, 'Sortino Ratio': nan, 'Max Drawdown': 0.0}


/tmp/ipykernel_4975/230901958.py:153: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  portfolio_value_df['daily_return'] = portfolio_value_df['portfolio_value'].pct_change().fillna(0)
/tmp/ipykernel_4975/230901958.py:155: RuntimeWarning: invalid value encountered in scalar divide
  sharpe_ratio = (portfolio_value_df['daily_return'].mean() / portfolio_value_df['daily_return'].std()) * np.sqrt(252)


# as of 14OCT24, realized this is the wrong format for a file like this. Moved to backtest.py

In [11]:
trade_log.to_csv('trade_log.csv', index=False)
# Need to add logic to close all positions at the end of the backtest
# for each trade, note if stopped out as well
# add a column for the trade duration
# add a column for rolling portfolio value

In [13]:
print(test_data.tail())

                     ts_event  rtype  publisher_id  instrument_id   open  \
245 2023-12-22 00:00:00+00:00     35             2          14941  69.77   
246 2023-12-26 00:00:00+00:00     35             2          14941  69.21   
247 2023-12-27 00:00:00+00:00     35             2          14941  69.45   
248 2023-12-28 00:00:00+00:00     35             2          14941  69.38   
249 2023-12-29 00:00:00+00:00     35             2          14941  69.79   

      high     low   close  volume symbol                      date  
245  70.20  69.335  69.455  425671     SO 2023-12-22 00:00:00+00:00  
246  69.83  69.170  69.450  259301     SO 2023-12-26 00:00:00+00:00  
247  69.66  69.020  69.410  320898     SO 2023-12-27 00:00:00+00:00  
248  70.32  69.280  70.170  371126     SO 2023-12-28 00:00:00+00:00  
249  70.25  69.630  70.120  337896     SO 2023-12-29 00:00:00+00:00  


In [14]:
print(y_pred)


[-8.08836333e-03 -8.08836333e-03 -8.08836333e-03 -8.08836333e-03
 -8.08836333e-03 -8.08836333e-03 -8.08836333e-03 -8.08836333e-03
 -8.08836333e-03 -8.08836333e-03 -8.08836333e-03 -8.08836333e-03
 -9.07882676e-03 -1.90817919e-02 -5.11187408e-03  3.73533927e-03
  3.55685683e-04 -8.94555170e-03 -5.88748325e-03  1.17391599e-02
  2.22211075e-03 -2.06932873e-02 -5.61465509e-03 -9.84477112e-04
 -2.64076646e-02 -3.60762514e-02  7.78150978e-03  2.72616148e-02
 -5.05026430e-03 -4.02532285e-03 -1.61050837e-02  1.31400917e-02
 -1.87743288e-02 -1.94315538e-02  8.57295748e-03  1.30745291e-03
 -2.51794364e-02 -4.45407778e-02 -1.79926790e-02  1.94552541e-02
  1.79682877e-02  9.63837747e-03 -7.19479565e-03 -6.01667538e-03
  8.08004756e-03 -6.08328078e-03  7.62430765e-03  1.75425969e-02
  1.88987870e-02  9.87434201e-03 -5.49496571e-03  1.21233100e-03
 -1.46229174e-02 -7.40941241e-03 -4.62152064e-03  1.74669661e-02
  2.82894005e-03  3.58172273e-03  1.19408509e-02  4.42109304e-04
  5.19943936e-03 -2.10564

In [15]:
print(test_data[['date', 'open', 'close']].iloc[i:i+5])


                         date   open  close
247 2023-12-27 00:00:00+00:00  69.45  69.41
248 2023-12-28 00:00:00+00:00  69.38  70.17
249 2023-12-29 00:00:00+00:00  69.79  70.12


In [16]:
dtest, y_test, X_test = prepdata(ticker, '2023-01-01', '2023-12-31')
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


X_test shape: (249, 74), y_test shape: (249,)


In [17]:
print(test_data[['date', 'open', 'close']].tail())


                         date   open   close
245 2023-12-22 00:00:00+00:00  69.77  69.455
246 2023-12-26 00:00:00+00:00  69.21  69.450
247 2023-12-27 00:00:00+00:00  69.45  69.410
248 2023-12-28 00:00:00+00:00  69.38  70.170
249 2023-12-29 00:00:00+00:00  69.79  70.120


In [18]:
print(test_data[['date']].min(), test_data[['date']].max())  # Check the date range before conversion to DMatrix.


date   2023-01-03 00:00:00+00:00
dtype: datetime64[ns, UTC] date   2023-12-29 00:00:00+00:00
dtype: datetime64[ns, UTC]


# Model Training

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from xgboost import XGBRegressor
from tqdm import tqdm  # Import tqdm for progress bar

# Combine training data for multiple tickers
tickers = ticker_list #tickerlist defined at the top of file
X_combined = []
y_combined = []

# Loop through tickers with progress bar
for ticker in tqdm(tickers, desc="Processing tickers"):
    dtrain, y_train, X_train = prepdata(ticker, '2019-01-01', '2022-12-31')
    X_combined.append(X_train)
    y_combined.append(y_train)

X_combined = pd.concat(X_combined)
y_combined = np.concatenate(y_combined)

# Initialize model and parameter grid
#xgb_regressor = XGBRegressor()
xgb_regressor = XGBRegressor(tree_method="hist", device="cuda") #GPu accelerated

#xgb_regressor.set_params(predictor="gpu_predictor") #GPu accelerated

param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7, 9],
    'n_estimators': [100, 200, 500],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'alpha': [0.1, 1, 10],
    'lambda': [0.1, 1, 10],
    'gamma': [0, 0.1, 0.5],
    'min_child_weight': [1, 5, 10],
}

# Use TimeSeriesSplit for time-based cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Perform GridSearchCV with cross-validation
grid_search = GridSearchCV(estimator=xgb_regressor, param_grid=param_grid, cv=tscv, scoring='r2', verbose=1)
grid_search.fit(X_combined, y_combined)

# Print the best parameter  s and R² score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best R² Score: {grid_search.best_score_}")


In [7]:
import cupy as cp
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from tqdm import tqdm  # Import tqdm for progress bar

# Combine training data for multiple tickers
tickers = ticker_list  # Define ticker_list at the top of your file
X_combined = []
y_combined = []

# Loop through tickers with a progress bar
for ticker in tqdm(tickers, desc="Processing tickers"):
    dtrain, y_train, X_train = prepdata(ticker, '2019-01-01', '2022-12-31')  # Assumes prepdata is defined
    # Convert to NumPy, then to CuPy arrays
    X_combined.append(cp.array(X_train.to_numpy()))  
    y_combined.append(cp.array(y_train.to_numpy()))

# Concatenate CuPy arrays
X_combined = cp.concatenate(X_combined)
y_combined = cp.concatenate(y_combined)

# Initialize model and parameter grid with GPU support using 'device' parameter
xgb_regressor = XGBRegressor(tree_method="hist", device="cuda")  # Updated for GPU support

# Set parameter grid for GridSearchCV
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7, 9],
    'n_estimators': [100, 200, 500],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'alpha': [0.1, 1, 10],
    'lambda': [0.1, 1, 10],
    'gamma': [0, 0.1, 0.5],
}

# Use TimeSeriesSplit for time-based cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Convert CuPy arrays to NumPy arrays for scikit-learn compatibility
X_combined_np = cp.asnumpy(X_combined)
y_combined_np = cp.asnumpy(y_combined)

# Perform GridSearchCV with cross-validation
grid_search = GridSearchCV(estimator=xgb_regressor, param_grid=param_grid, cv=tscv, scoring='r2', verbose=1)
grid_search.fit(X_combined_np, y_combined_np)

# Print the best parameters and R² score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best R² Score: {grid_search.best_score_}")


GridSearch Progress:   0%|          | 0/131220 [00:00<?, ?it/s]

Fitting 5 folds for each of 26244 candidates, totalling 131220 fits


GridSearch Progress:   0%|          | 0/131220 [00:09<?, ?it/s]


KeyboardInterrupt: 

In [27]:
#cupy v2 with progress bar
import cupy as cp
from xgboost import XGBRegressor
from tqdm import tqdm
from sklearn.model_selection import TimeSeriesSplit

# Combine training data for multiple tickers
tickers = ticker_list  # Define ticker_list at the top of your file
X_combined = []
y_combined = []

# Loop through tickers with a progress bar
for ticker in tqdm(tickers, desc="Processing tickers"):
    dtrain, y_train, X_train = prepdata(ticker, '2019-01-01', '2022-12-31')  # Assumes prepdata is defined
    # Convert to CuPy arrays directly
    X_combined.append(cp.array(X_train.to_numpy()))  
    y_combined.append(cp.array(y_train.to_numpy()))

# Concatenate CuPy arrays
X_combined = cp.concatenate(X_combined)
y_combined = cp.concatenate(y_combined)

# Initialize model with GPU support using the 'device' parameter
xgb_regressor = XGBRegressor(tree_method="hist", device="cuda")

# Set parameter grid for GridSearchCV
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7, 9],
    'n_estimators': [100, 200, 500],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'alpha': [0.1, 1, 10],
    'lambda': [0.1, 1, 10],
    'gamma': [0, 0.1, 0.5]
}

# Use TimeSeriesSplit for time-based cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Perform cross-validation using custom progress bar
total_fits = len(param_grid['learning_rate']) * len(param_grid['max_depth']) * len(param_grid['n_estimators']) \
    * len(param_grid['subsample']) * len(param_grid['colsample_bytree']) * len(param_grid['alpha']) \
    * len(param_grid['lambda']) * len(param_grid['gamma'])

with tqdm(total=total_fits, desc="XGBoost CV Progress") as pbar:
    best_score = float('-inf')
    best_params = None

    # Loop through the parameter grid
    for learning_rate in param_grid['learning_rate']:
        for max_depth in param_grid['max_depth']:
            for n_estimators in param_grid['n_estimators']:
                for subsample in param_grid['subsample']:
                    for colsample_bytree in param_grid['colsample_bytree']:
                        for alpha in param_grid['alpha']:
                            for lambd in param_grid['lambda']:
                                for gamma in param_grid['gamma']:
                                        # Define current parameter set
                                        params = {
                                            'learning_rate': learning_rate,
                                            'max_depth': max_depth,
                                            'subsample': subsample,
                                            'colsample_bytree': colsample_bytree,
                                            'alpha': alpha,
                                            'lambda': lambd,
                                            'gamma': gamma
                                        }

                                        # Apply parameters to the regressor
                                        xgb_regressor.set_params(**params)

                                        # Perform cross-validation using GPU
                                        xgb_regressor.fit(cp.asnumpy(X_combined), cp.asnumpy(y_combined))

                                        # Get score (you can adjust the scoring metric based on your use case)
                                        score = xgb_regressor.score(cp.asnumpy(X_combined), cp.asnumpy(y_combined))

                                        if score > best_score:
                                            best_score = score
                                            best_params = params

                                        pbar.update(1)

# Print the best parameters and score
print(f"Best Parameters: {best_params}")
print(f"Best R² Score: {best_score}")



XGBoost CV Progress: 100%|██████████| 8748/8748 [48:46<00:00,  2.99it/s]  

Best Parameters: {'learning_rate': 0.1, 'max_depth': 9, 'subsample': 1.0, 'colsample_bytree': 1.0, 'alpha': 0.1, 'lambda': 0.1, 'gamma': 0}
Best R² Score: 0.9469016867570385


In [ ]:
# Using CUDF and tqdm callbacks for accelerated gpu processing and progress tracking

import cudf
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from tqdm import tqdm

# Combine training data for multiple tickers
tickers = ticker_list  # Define ticker_list at the top of your file
X_combined = []
y_combined = []

# Loop through tickers with a progress bar
for ticker in tqdm(tickers, desc="Processing tickers"):
    dtrain, y_train, X_train = prepdata(ticker, '2019-01-01', '2022-12-31')  # Assumes prepdata is defined
    # Convert pandas DataFrame to cudf DataFrame
    X_combined.append(cudf.DataFrame.from_pandas(X_train))
    y_combined.append(cudf.Series.from_pandas(y_train))

# Concatenate cudf DataFrames
X_combined = cudf.concat(X_combined)
y_combined = cudf.concat(y_combined)

# Initialize model and parameter grid with GPU support using 'device' parameter
xgb_regressor = xgb.XGBRegressor(tree_method="hist", device="cuda")  # Updated for GPU support

# Set parameter grid for GridSearchCV
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7, 9],
    'n_estimators': [100, 200, 500],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'alpha': [0.1, 1, 10],
    'lambda': [0.1, 1, 10],
    'gamma': [0, 0.1, 0.5],
    'min_child_weight': [1, 5, 10],
}

# Use TimeSeriesSplit for time-based cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Convert cudf DataFrame to NumPy arrays for compatibility with scikit-learn
X_combined_np = X_combined.to_pandas().to_numpy()
y_combined_np = y_combined.to_pandas().to_numpy()

# Add a progress bar for GridSearchCV fits
n_candidates = len(param_grid['learning_rate']) * len(param_grid['max_depth']) * len(param_grid['n_estimators']) \
    * len(param_grid['subsample']) * len(param_grid['colsample_bytree']) * len(param_grid['alpha']) \
    * len(param_grid['lambda']) * len(param_grid['gamma']) * len(param_grid['min_child_weight'])

with tqdm(total=n_candidates * 5, desc="GridSearch Progress") as pbar:  # 5 = number of folds in TimeSeriesSplit
    class TQDMCallback(xgb.callback.TrainingCallback):
        def after_iteration(self, model, epoch, evals_log):
            pbar.update(1)
            return False  # Continue training

    # Perform GridSearchCV with cross-validation
    grid_search = GridSearchCV(estimator=xgb_regressor, param_grid=param_grid, cv=tscv, scoring='r2', verbose=1)
    grid_search.fit(X_combined_np, y_combined_np, callbacks=[TQDMCallback()])

# Print the best parameters and R² score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best R² Score: {grid_search.best_score_}")


In [ ]:
import itertools
import cupy as cp
from xgboost import XGBRegressor
from tqdm import tqdm
from sklearn.model_selection import TimeSeriesSplit

def perform_grid_search(X, y, param_grid, model):
    param_combinations = list(itertools.product(*param_grid.values()))
    best_score = float('-inf')
    best_params = None
    
    with tqdm(total=len(param_combinations), desc="XGBoost CV Progress") as pbar:
        for param_set in param_combinations:
            params = dict(zip(param_grid.keys(), param_set))
            model.set_params(**params)
            
            model.fit(cp.asnumpy(X), cp.asnumpy(y))
            score = model.score(cp.asnumpy(X), cp.asnumpy(y))
            
            if score > best_score:
                best_score = score
                best_params = params
            
            pbar.update(1)
    
    return best_params, best_score

# Example usage:
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7, 9],
    'n_estimators': [100, 200, 500],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'alpha': [0.1, 1, 10],
    'lambda': [0.1, 1, 10],
    'gamma': [0, 0.1, 0.5],
    'min_child_weight': [1, 5, 10],
}

# Perform the search
best_params, best_score = perform_grid_search(X_combined, y_combined, param_grid, xgb_regressor)
print(f"Best Parameters: {best_params}")
print(f"Best R² Score: {best_score}")


In [28]:
# After gridsearchiing using one of the above snippets:
# Create a new model using the best parameters
best_xgb_model = xgb.XGBRegressor(**best_params, tree_method="hist", device="cuda")

# Train the model using the training data
best_xgb_model.fit(X_combined_np, y_combined_np)

# Optionally, save the trained model to a file
import joblib
joblib.dump(best_xgb_model, 'best_xgb_model.pkl')

['best_xgb_model.pkl']

In [14]:
import joblib

# Assuming your model is the best one after grid search
best_model = xgb_regressor

# Save the best model
#best_model = grid_search.best_estimator_
joblib.dump(best_model, 'xgb_best_model.pkl')


['xgb_best_model.pkl']

In [ ]:
# Load the saved model
best_model = joblib.load('xgb_best_model.pkl')


In [29]:
# Test the model on new tickers

# Load the saved model
best_model = joblib.load('best_xgb_model.pkl')

for ticker in ticker_list:
    # Prepare data for the new ticker
    dtrain, y_train, X_train = prepdata(ticker, '2019-01-01', '2022-12-31')
    
    # Use the saved model to make predictions
    y_pred = best_model.predict(X_train)
    
    # Evaluate model performance
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    
    mae = mean_absolute_error(y_train, y_pred)
    mse = mean_squared_error(y_train, y_pred)
    r2 = r2_score(y_train, y_pred)
    
    print(f"Performance on {ticker}:")
    print(f"MAE: {mae}")
    print(f"MSE: {mse}")
    print(f"R²: {r2}")
    print('---')

Performance on AAPL:
MAE: 0.006129980545684062
MSE: 6.707176819986686e-05
R²: 0.9758889986845886
---
Performance on MSFT:
MAE: 0.005769208710626976
MSE: 6.284043404517403e-05
R²: 0.8467398029675239
---
Performance on NVDA:
MAE: 0.008611918454611345
MSE: 0.0001288901994853895
R²: 0.9644950916256266
---
Performance on AMZN:
MAE: 0.006882658121104334
MSE: 9.130347443430085e-05
R²: 0.992079503025956
---
Performance on TSLA:
MAE: 0.010956372819544449
MSE: 0.0002138563780594483
R²: 0.9660448994913136
---
Performance on HD:
MAE: 0.005488003530302216
MSE: 5.5421046097711826e-05
R²: 0.8608378271165196
---
Performance on UNH:
MAE: 0.005365407206637782
MSE: 5.760203111589428e-05
R²: 0.8454586279588304
---
Performance on JNJ:
MAE: 0.004026033129866135
MSE: 2.9471292142832132e-05
R²: 0.84006134424163
---
Performance on LLY:
MAE: 0.005684191315116926
MSE: 6.226316987209641e-05
R²: 0.8622775402991097
---
Performance on JPM:
MAE: 0.005824185272330074
MSE: 6.48761433455416e-05
R²: 0.8622051216597152
--

# Temp methods for troubleshooting

In [7]:
# Backtester get_data function
from datetime import datetime
# Load your price data for 2022 and 2023
def get_data(ticker, start_date, end_date):
    """
    Fetch data from PostgreSQL; if missing, download from Databento.
    """
    # Check if data already exists in PostgreSQL
    df = dbs.get_data_from_postgresql(ticker, start_date, end_date)  # Assuming this method exists in databento_sql.py

    if df is None or df.empty:
        # If data is missing, download from Databento
        print(f"Data for {ticker} not found in PostgreSQL. Fetching from Databento.")
        # Convert start_date and end_date to datetime objects
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
        df = dbs.get_data_from_databento(ticker, start_date, end_date)  # Download from Databento and return DataFrame

        # Upload the data to PostgreSQL for future use
        dbs.upload_to_postgresql(df, ticker)

    # Ensure the 'date' column is datetime and sorted by date
    df['date'] = pd.to_datetime(df['ts_event'])
    df = df.sort_values(by='date')

    return df

train_data = get_data('AAPL', '2019-01-01', '2022-12-31')

In [8]:
import os
from sqlalchemy import create_engine
from datetime import datetime
import pandas as pd

def get_data_from_postgresql(ticker, start_date=None, end_date=None, schema='databento_ohlcv'):
    """
    Retrieves data for a given ticker from PostgreSQL database, optionally within a date range.
    """
    # Fetch credentials from environment variables
    pguser = os.getenv('pguser')
    pgpass = os.getenv('pgpass')
    pghost = os.getenv('pghost')

    # Database connection URL using environment variables
    db_url = f'postgresql://{pguser}:{pgpass}@{pghost}/FinancialData'
    engine = create_engine(db_url)

    try:
        # Build the query
        query = f'SELECT * FROM "{schema}"."{ticker}"'
        if start_date is not None and end_date is not None:
            # Ensure start_date and end_date are strings

            # Convert date string to datetime object (if further handling is needed)
            start_date = datetime.strptime(start_date, '%Y-%m-%d')
            end_date = datetime.strptime(end_date, '%Y-%m-%d')

            query += f" WHERE ts_event BETWEEN '{start_date.strftime('%Y-%m-%d')}' AND '{end_date.strftime('%Y-%m-%d')}'"

        df = pd.read_sql(query, con=engine)

        # Ensure ts_event is parsed as timezone-aware datetime
        df['ts_event'] = pd.to_datetime(df['ts_event'], utc=True)

        return df
    except Exception as e:
        print(f"Error retrieving data for {ticker} from PostgreSQL: {e}")
        return None
    finally:
        engine.dispose()
train_data = get_data_from_postgresql('AAPL', '2019-01-01', '2022-12-31')

In [ ]:
train_data